In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import joblib
import datetime
import math
from datetime import datetime, timedelta
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense
from warnings import filterwarnings
import ccxt 

In [2]:
#แปลงข้อมูลจาก CCXT เป็น pandas DataFrame

ETH_data = ccxt.binance().fetch_ohlcv('ETH/USDT', timeframe='1d')
df = pd.DataFrame(ETH_data, columns=['Date', 'Open', 'High', 'Low', 'Close', 'Volume'])
df['Date'] = pd.to_datetime(df['Date'], unit='ms')
df['Date'] = df['Date'].dt.date
df.set_index('Date', inplace=True)

df.index[0]


datetime.date(2022, 10, 20)

In [3]:
def reshape_data(df, time_steps):
    
    X = []
    y = []

    for i in range(time_steps, len(df)):
        X.append(df[i-time_steps:i,0])
        y.append(df[i,0])
        
    return np.array(X), np.array(y)

In [4]:
def predict_values_for_future_dates(model, data, start_date, num_dates, time_steps):
    predictions = []

    current_date = datetime.combine(start_date, datetime.min.time())
    
    for _ in range(num_dates):
        input_data = data[-time_steps:].values
        input_data = input_data.reshape(1, time_steps, 1)
        
        prediction = model.predict(input_data)
        predictions.append(prediction[0, 0])
        
        current_date += timedelta(days=1)
        
        data = pd.concat([data, pd.DataFrame({'close': prediction[0, 0]}, index=[current_date])])

    return predictions

In [5]:
early_stop = EarlyStopping(monitor='loss', patience=5, verbose=1)
time_steps = 20
data = df.filter(['Close'])
dataset = data.values

training_data_len = math.ceil(len(dataset) * .8)
train_data = dataset[:training_data_len, :]
test_data = dataset[training_data_len-time_steps:, :]
train_dates = data[:training_data_len].index
test_dates = data[training_data_len-time_steps:].index
scaler = MinMaxScaler()

train_data = scaler.fit_transform(train_data)
test_data = scaler.transform(test_data)
X_train, y_train = reshape_data(train_data, time_steps)
X_test, y_test = reshape_data(test_data, time_steps)


X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))


In [6]:
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

In [7]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(380, 20, 1)
(100, 20, 1)
(380,)
(100,)


In [8]:
model = Sequential([
    LSTM(40, return_sequences=True, input_shape=(X_train.shape[1], 1)),
    Dropout(0.20),
    LSTM(40, return_sequences=False),
    Dropout(0.20),
    Dense(16),
    Dropout(0.20),
    Dense(1)
])

c:\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [9]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 20, 40)         │         6,720 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 20, 40)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 40)             │        12,960 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 40)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 16)             │           656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 20,353 (79.50 KB)

 Trainable params: 20,353 (79.50 KB)

 Non-trainable params: 0 (0.00 B)

In [10]:
model.compile(optimizer='adam', 
              loss='mean_squared_error', 
              metrics = ['mean_absolute_error']
             )

In [11]:
history = model.fit(
    X_train,
    y_train,
    callbacks=early_stop,
    epochs=50,
    batch_size=32,
    verbose=1
)

Epoch 1/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - loss: 0.4011 - mean_absolute_error: 0.5508
Epoch 2/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0458 - mean_absolute_error: 0.1770
Epoch 3/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0377 - mean_absolute_error: 0.1552
Epoch 4/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0317 - mean_absolute_error: 0.1435
Epoch 5/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0244 - mean_absolute_error: 0.1220
Epoch 6/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0249 - mean_absolute_error: 0.1218
Epoch 7/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0240 - mean_absolute_error: 0.1233
Epoch 8/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0255 - mean_absolute_error: 0.1316
Epoch 9/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0205 - mean_absolute_error: 0.1114
Epoch 10/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0209 - mean_absolute_error: 0.1131
Epoch 11/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 

In [12]:
model.evaluate(X_test, y_test)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0655 - mean_absolute_error: 0.1770  


[0.11023201793432236, 0.2029697299003601]

In [13]:
losses = pd.DataFrame(history.history)
losses.head()

,loss,mean_absolute_error
0,0.247742,0.401606
1,0.040173,0.162738
2,0.037437,0.154421
3,0.031003,0.144226
4,0.025056,0.124095


In [14]:
y_pred_train = model.predict(X_train)
y_pred_train = scaler.inverse_transform(y_pred_train)

y_train_normal = y_train.reshape((y_train.shape[0], -1))
y_train_normal = scaler.inverse_transform(y_train_normal)

12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step


In [15]:
y_pred_test = model.predict(X_test)
y_pred_test = scaler.inverse_transform(y_pred_test)

y_test_normal = y_test.reshape((y_test.shape[0], -1))
y_test_normal = scaler.inverse_transform(y_test_normal)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 


In [16]:
last_results = pd.DataFrame({'close': data.values.reshape(-1, )}, index=data.index)

In [17]:
last_results['close'] = scaler.transform(last_results['close'].values.reshape(-1, 1))

In [18]:
data.iloc[-1]


Close    3414.38
Name: 2024-03-02, dtype: float64

In [19]:
start_date = data.index[-1]
num_dates = 500
type(start_date)

datetime.date

In [20]:
p = predict_values_for_future_dates(model, last_results, start_date, num_dates+1, time_steps)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━

In [21]:
data.index[0]

datetime.date(2022, 10, 20)

In [22]:
NEW_DATES = [data.index[-1]]
for _ in range(num_dates):
    data_append = datetime.date(data.index[-1] + pd.DateOffset(days=_+1))
    NEW_DATES.append(data_append)




In [23]:
RESULTS = pd.DataFrame({'close': p[:]}, index=NEW_DATES)
RESULTS['close'] = scaler.inverse_transform(RESULTS[['close']])

In [24]:
model.save("eth_lstm_model.h5") # salvando o modelo
joblib.dump(scaler, 'eth_scaler.pkl')

['eth_scaler.pkl']

In [25]:
str(data.index[-1])

'2024-03-02'